# DeepMIR

## 1. Load tool packages

In [44]:
from Dataset import Dataset
import torch.utils.data as data
import torch
from DeepMIR import Model
import csv
from Metrics import Metrics

## 2. Load the model

In [45]:
model_path = '../model/DeepMIR.pth'
state_dict = torch.load(model_path)
model = Model()
model.load_state_dict(state_dict)
model.eval()

Model(
  (Conv1dA): Conv1d(1, 32, kernel_size=(7,), stride=(1,), padding=(3,))
  (BN1dA): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Act1dA): ReLU()
  (Pool1dA): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (Conv1dB): Conv1d(1, 32, kernel_size=(7,), stride=(1,), padding=(3,))
  (BN1dB): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Act1dB): ReLU()
  (Pool1dB): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (Conv2d1): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
  (BN2d1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Act2d1): ReLU()
  (Conv2d2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
  (BN2d2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Act2d2): ReLU()
  (Conv2d3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(4, 

## 3. Load the data

In [46]:
lib = ['1,2-dichloroethane', '1-butanol', 'acetonitrile', 'cyclohexane', 'dichloromethane', 'diethylene_glycol_dimethyl_ether', 'ethanol', 'hexane', 'isopropyl_alcohol', 'methanol', 'toluene', 'trichloromethane']
test_dataset = Dataset('../data/Quinary.npy')
test_dataloader = data.DataLoader(test_dataset, shuffle=False, batch_size=len(lib))

## 4. Component Prediction

In [47]:
csv_file = '../res/DeepMIR_Quinary.csv'
with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)

    for epoch in range(1):
        model.eval()

        for i, data in enumerate(test_dataloader, 0):
            inputs = data

            outputs = model(inputs)

            top_values, top_indices = torch.topk(outputs.view(-1), k=len(lib))

            # Prepare the row data as a list
            row_data = []
            for j in range(len(lib)):
                row_data.extend([lib[top_indices[j]], float(top_values[j])])

            # Write the row data to the CSV file
            writer.writerow(row_data)

## 5. Metrics

In [48]:
TPR, TNR, ACC = Metrics('../res/DeepMIR_Quinary.csv', '../data/Quinary_labels.csv')
print("TPR:{:.4f}".format(TPR), "TNR:{:.4f}".format(TNR), "ACC:{:.4f}".format(ACC))

TPR:0.8600 TNR:0.9714 ACC:0.9250
